## 환경설정


API KEY 를 설정합니다.


In [1]:
%pip install python-dotenv


[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다. 이러한 애플리케이션이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 [LangSmith](https://smith.langchain.com)를 사용하는 것입니다.

LangSmith가 필수는 아니지만, 유용합니다. LangSmith를 사용하고 싶다면, 위의 링크에서 가입한 후, 로깅 추적을 시작하기 위해 환경 변수를 설정해야 합니다.


In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("wemeet-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
wemeet-RAG


## 네이버 뉴스 기반 QA(Question-Answering) 챗봇

이번 튜토리얼에는 네이버 뉴스기사의 내용에 대해 질문할 수 있는 **뉴스기사 QA 앱** 을 구축할 것입니다.

이 가이드에서는 OpenAI 챗 모델과 임베딩, 그리고 Chroma 벡터 스토어를 사용할 것입니다.

먼저 다음의 과정을 통해 간단한 인덱싱 파이프라인과 RAG 체인을 약 20줄의 코드로 구현할 수 있습니다.

라이브러리

- `bs4`는 웹 페이지를 파싱하기 위한 라이브러리입니다.
- `langchain`은 AI와 관련된 다양한 기능을 제공하는 라이브러리로, 여기서는 특히 텍스트 분할(`RecursiveCharacterTextSplitter`), 문서 로딩(`WebBaseLoader`), 벡터 저장(`Chroma`, `FAISS`), 출력 파싱(`StrOutputParser`), 실행 가능한 패스스루(`RunnablePassthrough`) 등을 다룹니다.
- `langchain_openai` 모듈을 통해 OpenAI의 챗봇(`ChatOpenAI`)과 임베딩(`OpenAIEmbeddings`) 기능을 사용할 수 있습니다.


In [4]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


웹 페이지의 내용을 로드하고, 텍스트를 청크로 나누어 인덱싱하는 과정을 거친 후, 관련된 텍스트 스니펫을 검색하여 새로운 내용을 생성하는 과정을 구현합니다.

`WebBaseLoader`는 지정된 웹 페이지에서 필요한 부분만을 파싱하기 위해 `bs4.SoupStrainer`를 사용합니다.

[참고]

- `bs4.SoupStrainer` 는 편리하게 웹에서 원하는 요소를 가져올 수 있도록 해줍니다.

(예시)

```python
bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
)
```


In [ ]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
#loader = WebBaseLoader(
#    web_paths=("https://n.news.naver.com/article/437/0000378416",),
#    bs_kwargs=dict(
#        parse_only=bs4.SoupStrainer(
#            "div",
#            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
#        )
#    ),
#)

#docs = loader.load()
#print(f"문서의 수: {len(docs)}")
#docs

`RecursiveCharacterTextSplitter`는 문서를 지정된 크기의 청크로 나눕니다.


In [5]:
from langchain_community.document_loaders import TextLoader

# 인코딩을 'utf-8'로 지정하여 로더 생성
loader = TextLoader("data3/doc-01.txt", encoding='utf-8')
docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

# 10번째 페이지의 내용 출력
#print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
#print(f"\n[metadata]\n{docs[0].metadata}\n")

문서의 수: 1


[Document(metadata={'source': 'data3/doc-01.txt'}, page_content='소프트웨어 개발 표준계약서\n\n\n 과학기술정보통신부에서 보급하는 소프트웨어사업자 표준계약서 4종은 소프트웨어진흥법 제2조 제3호의 소프트웨어사업에 대해 소프트웨어진흥법 제38조(공정계약의 원칙)에 따라 계약당사자들이 동등한 지위에서 합의에 따라 공정하게 계약을 체결하고 신의에 따라 성실하게 계약을 이행할 수 있도록 마련되었습니다. 이 표준계약서는 도급계약인 소프트웨어 용역위탁사업에서 계약당사자 간의 기본적이고 원칙적인 사항만을 제시하였으며, 계약당사자는 이 표준계약서의 기본 틀과 내용을 유지하는 범위에서 보다 더 상세하고 개별적인 사항을 계약서에 규정할 수 있습니다. 또한 이 표준계약서의 일부 내용은 현행 소프트웨어진흥법, 민법, 공정거래법, 『하도급거래 공정화에 관한 법률』(이하 “하도급법”이라 한다) 및 그 시행령을 기준으로 마련됨에 따라, 계약당사자는 이들 법령이 개정될 경우 개정 내용에 부합되도록 표준 도급계약서의 내용을 수정 또는 변경하여 사용할 수 있습니다.\n\n 다만, 계약당사자 중 한 쪽이 소프트웨어진흥법의 국가기관등인 경우에는 국가계약법 또는 지방계약법이, 양 쪽이 하도급법 상의 원사업자와 수급사업자에 해당하는 경우에는 하도급법이 우선 적용됩니다. 그러므로 국가기관등인 경우에는 관련 법령을 준수하는 자체적인 소프트웨어사업계약서를 사용하여야 하며, 하도급법 상의 원사업자와 수급사업자는 공정거래위원회가 보급하는 소프트웨어사업 표준하도급계약서를 사용하여야 합니다.\n-표준계약서 마련 및 활용 취지-\n\n\nNo.\n개발 App의 종류\n개발비\n1\nAI번역회의록 Appp\n\\15,000,000\n2\n음성명령 App\n\\5,000,000\n3\n인터넷전화 App\n무상\n4\n화상회의 App(AI번역)\n무상\n1. 계약명 : AHA 전자칠판 전용 App 개발(4종)의 건\n\n2. 계약기간 : 2023년 7월 10일부터 

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

splits = text_splitter.split_documents(docs)
len(splits)

52

`FAISS` 혹은 `Chroma`와 같은 vectorstore는 이러한 청크를 바탕으로 문서의 벡터 표현을 생성합니다.


In [7]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

`vectorstore.as_retriever()`를 통해 생성된 검색기는 `hub.pull`로 가져온 프롬프트와 `ChatOpenAI` 모델을 사용하여 새로운 내용을 생성합니다.

마지막으로, `StrOutputParser`는 생성된 결과를 문자열로 파싱합니다.


In [8]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

hub 에서 `teddynote/rag-prompt-korean` 프롬프트를 다운로드 받아 입력할 수 있습니다. 이런 경우 별도의 프롬프트 작성과정이 생략됩니다.

In [ ]:
# prompt = hub.pull("teddynote/rag-prompt-korean")
# prompt

In [9]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

스트리밍 출력을 위하여 `stream_response` 를 사용합니다.

In [10]:
from langchain_teddynote.messages import stream_response

> [LangSmith Trace 보기](https://smith.langchain.com/public/c6047a61-8f44-48e5-89eb-b1e8a6321cea/r)


In [14]:
#answer = rag_chain.stream("부영그룹의 출산 장려 정책에 대해 설명해주세요.")
#stream_response(answer)

#answer = rag_chain.stream("계양구의 대형폐기불을 버리는 방법은?")

answer = rag_chain.stream("계약의 내용에서 계약의 목적물로 apk등 공급자가 발주자에게 제공해야 할 것들은?")
stream_response(answer)

계약의 목적물로 공급자가 발주자에게 제공해야 할 것들은 "AI번역회의록 App"과 "음성명령 App"의 기술자료 중 소스코드 및 Apk와 기술정보(메뉴얼, 설계서, 사양서, 플로우차트, 유지관리자료 등)입니다. 이들은 납품 시 제출해야 합니다.

> [LangSmith Trace 보기](https://smith.langchain.com/public/ed21d80e-b4da-4a08-823b-ed980db9c347/r)
